In [9]:
# 가로로 넓게 쓰기
from IPython.core.display import display, HTML  
display(HTML(""))

print("Setup Complete")

Setup Complete


In [10]:
import pandas as pd
import numpy as np
import os

import time 
import datetime as dt
from datetime import datetime

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('seaborn-whitegrid')

from sklearn.preprocessing import LabelEncoder

In [11]:
path = "/home/kym/ML/input/kamp/대회 과제"
os.chdir(path)

df = pd.read_csv('melting_tank_mod.csv', encoding='cp949')

feature = [
    f for f in df.columns if f not in ("NUM")
]

df = df[feature]
df['STD_DT'] = df['STD_DT'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d %H:%M:%S', errors='coerce'))
df.set_index('STD_DT', inplace=True)
df.to_csv('/home/kym/ML/input/kamp/대회 과제/melting_tank_final.csv', index=False)
df.head()

,STD_DT,MELT_TEMP,MOTORSPEED,MELT_WEIGHT,TAG
0,2020-03-04,48.9,11.6,631,1
1,2020-03-04,43.3,7.8,609,1
2,2020-03-04,46.4,15.4,608,1
3,2020-03-04,37.9,21.2,606,1
4,2020-03-04,79.8,173.6,604,1


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835200 entries, 0 to 835199
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   STD_DT       835200 non-null  datetime64[ns]
 1   MELT_TEMP    835200 non-null  float64       
 2   MOTORSPEED   835200 non-null  float64       
 3   MELT_WEIGHT  835200 non-null  int64         
 4   TAG          835200 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 31.9 MB


In [13]:
df.describe().T.style.bar(subset=['mean'], color='#205ff2').background_gradient(subset=['std'], cmap='Reds')

,count,mean,std,min,25%,50%,75%,max
MELT_TEMP,835200.000000,50.920062,12.827752,30.800000,43.000000,46.900000,50.200000,83.200000
MOTORSPEED,835200.000000,45.978287,63.943641,0.000000,11.900000,16.800000,21.800000,180.400000
MELT_WEIGHT,835200.000000,414.582481,317.234065,0.000000,186.000000,382.000000,579.000000,2820.000000
TAG,835200.000000,0.787994,0.408729,0.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
X = df.drop(['STD_DT', 'TAG'], axis=1)
y = df['TAG']

In [17]:
# TPOT : Automatic ML library that automatically enables feature selection and Hyperparameter learning based on multiple machine learning models 
from tpot import TPOTClassifier

reference site : https://machinelearningmastery.com/tpot-for-automated-machine-learning-in-python/

In [23]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV, GroupKFold

# ML Library 
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_absolute_error

In [24]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [25]:
tpotc = TPOTClassifier(
                        generations=100, 
                        population_size=100, 
                        scoring=['r2', 'accuracy'], 
                        cv=5,
                        verbosity=1, 
                        n_jobs=-1, 
                        random_state=42, 
                        config_dict='TPOT light')
tpotc.fit(x_train, y_train)



TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
Best pipeline: DecisionTreeClassifier(DecisionTreeClassifier(MaxAbsScaler(input_matrix), criterion=gini, max_depth=7, min_samples_leaf=5, min_samples_split=7), criterion=entropy, max_depth=8, min_samples_leaf=4, min_samples_split=10)


TPOTClassifier(config_dict='TPOT light', n_jobs=-1, random_state=42,
               scoring=['r2', 'accuracy'], verbosity=1)

In [26]:
print("\nBest Pipeline steps:", end='\n')

for idx, (name, transform) in enumerate(tpotc.fitted_pipeline_.steps, start=1):
    print(f"{idx}. {transform}")

tpotc.fitted_pipeline_.steps[-1][1] # get the best model


Best Pipeline steps:
1. MaxAbsScaler()
2. StackingEstimator(estimator=DecisionTreeClassifier(max_depth=7,
                                                   min_samples_leaf=5,
                                                   min_samples_split=7,
                                                   random_state=42))
3. DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=4,
                       min_samples_split=10, random_state=42)


DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=4,
                       min_samples_split=10, random_state=42)

In [28]:
print(tpotc.score(x_test, y_test))
tpotc.export('tpot_exported_pipeline.py')

0.7682590996168582


/home/kym/miniconda3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


In [ ]:
time_lag = 10